
# Lab 1 – Insamling av data (IoT → CSV/JSON)

Länk till instruktioner :

https://www.notion.so/Scenario-Smart-Home-Energy-Analytics-2a79de1f0d8380dd926bf8965697d9c0?source=copy_link


##Del A - I den första delen jobbar vi med en liten exempel datamängd i form av csv data/mock data, som har bara 31 datapunkter

In [ ]:
# klona ditt repo (din fork) till Colab
!git clone https://github.com/<DITT-ANVÄNDARNAMN>/<REPO-NAMN>.git

# visa filerna i repot
!ls la

# gå till rätt mapp, lab1-datainsamling
%cd lab1-datainsamling/data
!ls la

### Härifrån förljer du från steg 4 i den här guiden:
https://github.com/maryam-marzban-it/big-data-iot-labs/blob/main/Startguide_Git_Colab_Fork.ipynb

In [ ]:
# För att läsa in övningsdata.csv, import panda
import panda as pd

# Läser in datafilen med hjälp av pd
fittness = pd.read_csv("övningsdata.csv")

# Laddar csv filen i en Dataframe

df_home_fittness = pd.DataFrame(fittness)

# Nu kan vi hantera, analysera och manipulera våra dataframe

df_home_fittness.head() # ger oss de första fem raderna av data
df_home_fittness.tail() # ger oss sista raderna
df_home_fittness.shape  # formen på dataframe
df_home_fittness.size   # storleken på dataframe
df_home_fittness.columns # ger columner


In [ ]:
# Visar vilka datatyper vår dataframe innehåller
df_home_fittness.info()

# Visar statistik
df_home_fittness.describe()

# Sample ger oss random data
df_home_fittness.sample(10)


In [ ]:
# loc ger oss filter data,
df_home_fittness.loc[1,2]  # visar andra och tredje raden i dataframe
df_home_fittness.loc[1:5]  # visar rad 1 till 5
df_home_fittness.loc[1:]   # visar rad 1 och alla rader efter 1
df_home_fittness.loc[4:20,["Calories_Burned"]] # visar Calories_Burned för rad 4 till 20


In [ ]:
df_home_fittness.head()

# om vi vill ändra value för en specifik kolumn
df_home_fittness.loc[10,"Calories_Burned"] = 400

# vi kan komma åt datan i en kolumn
df_home_fittness["Calories_Burned"]

# visa och sortera värden i en kolumn
df_home_fittness.sort_value("Calories_Burned") # visar värden i ökande order
df_home_fittness.sort_value("Calories_Burned", ascending =False) # visar värden i fallande order

# visa värden för en kolumn som följer viss regel
selected_rows = df_home_fittness.loc[df_home_fittness["Calories_Burned"] < 500]


## Del B – I denna del jobbar vi med riktiga Big Data, hämtat från kaggle datasets


https://www.kaggle.com/datasets


Datat ligger under data mappen på Github

https://github.com/maryam-marzban-it/big-data-iot-labs/blob/main/lab1-datainsamling/data/smart_home_energy_consumption_large.csv




### Övning B1 – Förstå datan
1. Vilka kolumner finns och vad betyder de?  
2. Vilken tidsupplösning har datan?  
3. Finns det saknade värden?
4. Vilka kolumner verkar viktigast för att analysera energiförbrukningen?

In [ ]:
# vi läser in csv filen
df_home_home = pd.read_csv("raw_smart_home_data.csv")

df_home_home.size

# kollar på första raderna
df_home_home.head()




Struktur & metadata

In [ ]:
# datatyper,minne och nullvärden
df_home_home.info()

# statistik sammanfattning
df_home_home.describe()


# antal rader och kolumner
df_home_home.shape

# kolumnnamn
df_home_home.columns

In [ ]:
# koden printar en sample på 3 slumpmässiga datapunkter
print("\nExempelrader:\n", df_home_home.sample(3))

Datakvalitet

In [ ]:
# tittar på datakvalitet

df_home_home.isnull().sum() # hittar saknade värden
df_home_home.duplicated().sum() # hittar dubbletter


In [ ]:
# här printas först medelvärden på energyförbrukning
print(df_home['Energy Consumption (kWh)'].mean().round(2))

# min och max värden
print(df_home['Energy Consumption (kWh)'].min(), df_home['Energy Consumption (kWh)'].max())

In [ ]:

if isinstance(df_home_home, pd.DataFrame):
    display(df_home_home.info())
    display(df_home_home.isna().sum())


In [ ]:
# med hjälp av plot kan vi visa grafisk data
df_home.head(500)['Energy Consumption (kWh)'].plot(kind='line',title='Energiförbrukning över tid')

In [ ]:
# genomsnitt per timme
hourly_avg = df_home.head(500).groupby('Time')['Energy Consumption (kWh)'].mean()
hourly_avg.plot(kind='line', marker='o')

In [ ]:
# visar där det saknas data
print("Saknade värden per kolumn:")
print(df_home.isna().sum())

#visar dupliceringar
print("Antal dubbletter:", df_home.duplicated().sum())

In [ ]:
# visar outliner-värden som avviker kraftig från resten och std som betyder hut mycket mätvärdena varierar från genomsnittet
# en stor std betyder att spridningen mellan värdena är stor och mätningar är ojämna
outliers = df_home[(df_home['Energy Consumption (kWh)'] < 0) | (df_home['Energy Consumption (kWh)'] > 10000)]
print("Orimliga värden:", len(outliers))

print("Standardavvikelse (std):", df_home['Energy Consumption (kWh)'].std())

In [ ]:
#Här markerar vi punkter där skillnaden mot föregående mätning är onormalt stor — det är ofta brus i signalen
df_home['AbsDiff'] = df_home['Diff'].abs()
threshold = df_home['AbsDiff'].mean() + 3 * df_home['AbsDiff'].std()
noisy_points = df_home[df_home['AbsDiff'] > threshold]


### Övning B2 – Spara och ladda
- Spara datan till **CSV** och **JSON**.  
- Ladda tillbaka och kontrollera att innehållet stämmer.


In [ ]:

if isinstance(df_home_home, pd.DataFrame):
    df_home_home.to_csv("raw_iot_data.csv", index=False)
    df_home_home.to_json("raw_iot_data.json", orient="records")
    print("Filer sparade: raw_iot_data.csv, raw_iot_data.json")

    df_home_csv = pd.read_csv("raw_iot_data_api.csv")
    df_home_json = pd.read_json("raw_iot_data_api.json")
    print("CSV shape:", df_home_csv.shape, "JSON shape:", df_home_json.shape)


###Övning B3 - Utforska och förstå data

se mönster, variation och kvalitet – förstå datans tillstånd (är den rå eller bearbetad?)

In [ ]:
# Granska värden och variation
print("Kolumner:", df_home_home.columns.tolist())
print("\nExempelrader:\n", df_home_home.sample(3))

#medelvärde för energyförbrukning
print(df_home['energy_consumption'].mean())

# min och max värde
print(df_home['energy_consumption'].min(), df_home['energy_consumption'].max())

In [ ]:
# Visa distribution / trend
df_home['energy_consumption'].plot(kind='line', title='Energiförbrukning över tid')

df_home.head(500)['Energy Consumption (kWh)'].plot(kind='line', title='Energiförbrukning över tid')

hourly_avg = df_home.head(500).groupby('Time')['Energy Consumption (kWh)'].mean()
hourly_avg.plot(kind='line', marker='o')


In [ ]:
# Identifiera fel och problem
print("Saknade värden per kolumn:")
print(df_home.isna().sum())

print("Antal dubbletter:", df_home.duplicated().sum())

In [ ]:
# kolla orimliga värden
outliers = df_home[(df_home['Energy Consumption (kWh)'] < 0) | (df_home['Energy Consumption (kWh)'] > 10000)]
print("Orimliga värden:", len(outliers))



---

## ✅ Sammanfattning
- Du har hämtat en stor dataset  
- Du har sparat den i **CSV/JSON** och kontrollerat innehåll och kvalitet.  
  

**Inför nästa labb** förbereder vi **datarensning (data cleaning)** av filen du skapade idag.
